In [54]:
from typing import Optional, List
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import os
import codecs
from sklearn.linear_model import LogisticRegression
import numpy as np

In [200]:
class ZamokClassifier:
    def __init__(self, load_path: Optional[str] = None) -> None:
        if load_path:
            with open(load_path+'.vectorizer.pk', 'rb') as fin:
                self.vectorizer = pickle.load(fin)
            with open(load_path+'.model.pk', 'rb') as fin:
                self.log_model = pickle.load(fin)
        else:
            self.vectorizer = TfidfVectorizer(max_features=150)
            self.log_model = LogisticRegression(random_state=0)
            
    def preproc(self, texts):
        bad = '1234567890()!@#$%^&*,./abcdefghijklmnopqrstuvwxyz-—_+=«»\n:;[]–’…'+"'"+'"'
        clear_ = []
        for text in texts:
            text = text.lower()
            for letter in text:
                if letter in bad:
                    text = text.replace(letter, '')
                text = text.replace('  ', ' ')
            clear_.append(text)
        return clear_
            
    def train(self, texts1: List[str], texts2: List[str]) -> None:
        # castles_tokens = [token for text in texts1 for token in text.split()]
        # locks_tokens = [token for text in texts2 for token in text.split()]
        texts1, texts2 = self.preproc(texts1), self.preproc(texts2)
        self.vectorizer.fit(texts1+texts2)
        x_castles = self.vectorizer.transform(texts1).toarray()
        x_locks = self.vectorizer.transform(texts2).toarray()
        X = np.concatenate([x_castles, x_locks], axis=0)
        y = np.concatenate([[1]*len(x_castles), [0]*len(x_locks)])
        self.log_model.fit(X, y)
    
    def save(self, path: str) -> None:
        with open(path+'.vectorizer.pk', 'wb') as fin:
            pickle.dump(self.vectorizer, fin)
        with open(path+'.model.pk', 'wb') as fin:
            pickle.dump(self.log_model, fin)
        
    
    def predict(self, text: str) -> str:
        emb = self.vectorizer.transform(self.preproc([text]))
        class_ = self.log_model.predict(emb)
        if class_:
            ans = 'YES'
        else:
            ans = 'NO'
        return ans
        

In [201]:
model = ZamokClassifier()

In [202]:
castles = []
for filename in os.listdir("castles/"):
    if filename[::-1][:4] == 'txt.':
        fileObj = codecs.open("castles/"+filename, 'r', "utf_8_sig")
        txt = fileObj.readlines()
        castles.extend(txt)
        fileObj.close()
        
locks = []
for filename in os.listdir("locks/"):
    if filename[::-1][:4] == 'txt.':
        fileObj = codecs.open("locks/"+filename, 'r', "utf_8_sig")
        txt = fileObj.readlines()
        locks.extend(txt)
        fileObj.close()

In [203]:
castles[6]

'До 1975 года Шлоссбург был независимым городком, после чего вошёл в состав Золингена. В настоящее время замок является популярным туристическим местом. В замке действует музей графства Берг и находятся несколько исторических памятников разных эпох. С 2005 года на территории замка проводятся археологические раскопки.\n'

In [204]:
clear_castles[6]

'до года шлоссбург был независимым городком после чего вошёл в состав золингена в настоящее время замок является популярным туристическим местом в замке действует музей графства берг и находятся несколько исторических памятников разных эпох с года на территории замка проводятся археологические раскопки'

In [205]:
len(castles)

329

In [206]:
len(locks)

284

In [207]:
model.train(castles, locks)

In [210]:
model.log_model.coef_

array([[ 1.47146705,  1.49852991, -0.43268179, -1.46315173, -0.67001986,
        -0.09333417,  1.17991825,  0.43903362,  0.29182026, -0.88554092,
         1.39370126,  1.38799279, -1.52808731, -0.41449941,  1.21259757,
         0.88751932,  0.58907468,  0.34065751,  0.61040266,  0.04136747,
        -0.41334506, -0.16768609, -0.63947959,  1.1958363 ,  0.89419324,
         0.82043465,  0.64200376, -1.59875045, -1.01956019,  0.8164329 ,
         0.85559329, -1.08877767,  0.07433988, -0.98462946, -0.26692492,
        -0.53761964, -0.03508372, -0.98623311,  0.21492371,  1.08792729,
        -0.15702336, -0.30787961, -0.05913817, -0.60285069,  0.20976191,
        -1.91591691, -2.26664735, -0.41894241,  1.02626027,  0.91651415,
         0.21446142, -1.86866175,  0.71398211, -0.74235735,  0.57035985,
        -1.12538035, -1.19951466,  0.25948245, -0.30031672, -0.85642978,
         0.15222992, -2.41776558, -2.50228679, -1.12300061, -1.42385553,
        -1.07058487, -0.98263765, -0.99850865, -0.2

In [199]:
for i in range(len(locks)):
    if model.predict(locks[i]) == "YES":
        print("-")
        # print(locks[i])

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-


In [84]:
np.concatenate([x1,x2], axis=0).shape

(613, 100)

In [85]:
np.concatenate([[0]*len(x1), [1]*len(x2)]).shape

(613,)

In [86]:
model.save('test_model')

In [87]:
model1 = ZamokClassifier('test_model')

In [88]:
model1.predict(locks[101])

'NO'